# Instructions to get started
  1. Run the following commands
```python setup.py build```
```python setup.py install```
  4. Use `pip` to install any packages you don't already have (make sure you have pims, and pims_nd2)
  5. Unzip the neural network parameter files and put them somewhere 
  6. Alter the `modelPath` variable below to wherever you put the model files
  7. Alter the `vidPath` variable to the folder containing some ND2 videos you want to process

In [1]:
%pylab inline
%config InlineBackend.figure_format = 'retina'
from ipywidgets import interact
import ipywidgets as widgets
# import warnings
# warnings.filterwarnings("ignore")
from IPython.display import display, HTML
import os
import pandas as pd
import time
from io import BytesIO

import tensorflow as tf
import NetTracker as NT
import pims
import pims_nd2


modelPath = 'models/3L-RNN-3D-set1-EMA'
vidPath = '<video path>'


def getFiles(path, kind='nd2'):
    kind = '.'+kind
    fn = len(kind)
    filenames = []
    for root, dirs, files in os.walk(path):
        for f in files:
            if f[-fn:] == kind:
                filenames.append(os.path.join(root, f[:-fn]))
    print('Total number of {0} files: '.format(kind[1:]), len(filenames))
    return filenames 

Populating the interactive namespace from numpy and matplotlib


In [ ]:
filenames = getFiles(vidPath)
for fn in filenames[:1]:
    _, vidName = os.path.split(fn)
    VIDS = []
    with pims_nd2.ND2_Reader(fn + '.nd2') as frames:
        frames.bundle_axes = ['y', 'x', 'z', 'c']
        vid_all_colors = array(frames)
        vid1 = vid_all_colors[..., 2]
        vid2 = vid_all_colors[..., 3]
        Nt, Ny, Nx, Nz = vid1.shape
        shape = (Nt, Ny, Nx, Nz)
        VIDS.append({'videoData': vid1})
        VIDS.append({'videoData': vid2})
    for data, color_channel in zip(VIDS, ['_C3', '_C4']):
        ## first, get the mean and standard deviation which is used by the neural net
        vm = data['videoData'].mean(axis=(1, 2, 3))
        vs = (float64(data['videoData'])**2).mean(axis=(1, 2, 3))
        data['stats'] = array([vm, vs]).T.reshape(Nt, 1, 2)
        ## second, set various parameters, only two need to be changed (marked as "REQUIRED")
        data['metadata'] = {
            'fileName': vidName, ## don't change this
            'chunkIndex': (0, 0, 0, 0), ## don't change this
            'dt': 1, ## you don't need to set this, it doesn't do anything
            'dxy': 0.11, ## REQUIRED (if 2D video, can just set to 1)
            'dz': 0.3, ## REQUIRED (if 2D video, can just set to 1)
            'vidShape': shape ## don't change this
        }
        nn_obj = NT.NeuralNet()
        nn = nn_obj.process(('data', data), modelPath)
        seg_obj = NT.Segment()
        ## next() is a function used by advanced iterators to get the next element in a sequence, it 
        ## is used here because the code was originally written for an Apache Beam pipeline, which uses
        ## the fancy iterators
        seg = seg_obj.process(next(nn))
        _, seg_out = next(seg)
        df = seg_out['pointSet']
        # df.to_csv(fn + color_channel + '.csv')
